<a href="https://colab.research.google.com/github/Gauthampharish/OCR/blob/main/Untitled10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dense, Flatten, Softmax

print(tf.__version__)

2.14.0


In [41]:
!pip install emnist



In [42]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import emnist

In [43]:
from emnist import extract_training_samples
train_x, train_y = extract_training_samples('letters')

train_x = train_x /255.0
train_y = train_y - 1

In [44]:
from emnist import extract_test_samples
test_x, test_y = extract_test_samples('letters')
test_x = test_x / 255.0
test_y = test_y - 1

In [45]:
model = models.Sequential()

In [46]:
model = models.Sequential()

# Input layer
model.add(layers.Input(shape=(28, 28, 1)))

# Convolutional Layer 1
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# Convolutional Layer 2
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten Layer
model.add(layers.Flatten())

# Fully Connected Layer 1
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(128, activation='relu'))

# Dropout Layer
model.add(layers.Dropout(0.5))  # You can adjust the dropout rate as needed

# Output Layer
model.add(layers.Dense(26, activation='softmax'))  # Assuming 26 classes for the English alphabet

# Compile the model

# Print a summary of the model architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 3, 3, 128)        

In [47]:


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, validation_split = 0.2, batch_size = 30, epochs = 13)


Epoch 1/13
3328/3328 [==============================] - 241s 72ms/step - loss: 0.5141 - accuracy: 0.8408 - val_loss: 0.2269 - val_accuracy: 0.9204
Epoch 2/13
3328/3328 [==============================] - 235s 71ms/step - loss: 0.2406 - accuracy: 0.9246 - val_loss: 0.1944 - val_accuracy: 0.9356
Epoch 3/13
3328/3328 [==============================] - 234s 70ms/step - loss: 0.1984 - accuracy: 0.9362 - val_loss: 0.1918 - val_accuracy: 0.9389
Epoch 4/13
3328/3328 [==============================] - 219s 66ms/step - loss: 0.1723 - accuracy: 0.9415 - val_loss: 0.2027 - val_accuracy: 0.9389
Epoch 5/13
3328/3328 [==============================] - 212s 64ms/step - loss: 0.1551 - accuracy: 0.9478 - val_loss: 0.1911 - val_accuracy: 0.9419
Epoch 6/13
3328/3328 [==============================] - 211s 64ms/step - loss: 0.1400 - accuracy: 0.9512 - val_loss: 0.2017 - val_accuracy: 0.9440
Epoch 7/13
3328/3328 [==============================] - 218s 65ms/step - loss: 0.1290 - accuracy: 0.9540 - val_loss: 0

In [48]:
from tensorflow.keras.models import load_model, save_model

# Save the model
model.save("your_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
!git clone https://github.com/dilhelh/opencv-text-detection.git

fatal: destination path 'opencv-text-detection' already exists and is not an empty directory.


In [50]:
!python /content/opencv-text-detection/text_detection.py --image /content/opencv-text-detection/images/lebron_james.jpg \
	--east /content/opencv-text-detection/frozen_east_text_detection.pb

[INFO] loading EAST text detector...
[INFO] text detection took 0.923866 seconds
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.10/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [51]:
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

def east_detect(image):
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"]

    orig = image.copy()

    if len(image.shape) == 2:
        # Convert grayscale image to BGR by duplicating the channel
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    (H, W) = image.shape[:2]

    # set the new width and height and then determine the ratio in change
    # for both the width and height: Should be multiple of 32
    (newW, newH) = (320, 320)

    rW = W / float(newW)
    rH = H / float(newH)

    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))

    (H, W) = image.shape[:2]

    net = cv2.dnn.readNet(cv2.samples.findFile("/content/opencv-text-detection/frozen_east_text_detection.pb"))

    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H), (123.68, 116.78, 103.94), swapRB=True, crop=False)

    start = time.time()

    net.setInput(blob)

    (scores, geometry) = net.forward(layerNames)

    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the geometrical
        # data used to derive potential bounding box coordinates that
        # surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        for x in range(0, numCols):
            # if our score does not have sufficient probability, ignore it
            # Set minimum confidence as required
            if scoresData[x] < 0.5:
                continue
            # compute the offset factor as our resulting feature maps will
            #  x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            # extract the rotation angle for the prediction and then
            # compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            # use the geometry volume to derive the width and height of
            # the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            # compute both the starting and ending (x, y)-coordinates for
            # the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            # add the bounding box coordinates and probability score to
            # our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    boxes = non_max_suppression(np.array(rects), probs=confidences)

    # Initialize a list to store characters in MNIST-like format
    characters = []

    # Loop over the bounding boxes
    for (startX, startY, endX, endY) in boxes:
        # Check if the bounding box is valid and not empty
        if startX < endX and startY < endY:
            # Extract the region of interest (ROI) for each character
            roi = orig[startY:endY, startX:endX]

            # Check if the ROI is large enough to resize to 28x28
            if roi.shape[0] > 0 and roi.shape[1] > 0:
                # Resize the character to MNIST image size (28x28)
                mnist_character = cv2.resize(roi, (28, 28))

                # Convert to grayscale
                mnist_character = cv2.cvtColor(mnist_character, cv2.COLOR_BGR2GRAY)

                # Apply thresholding to binarize the image
                _, mnist_character = cv2.threshold(mnist_character, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

                characters.append(mnist_character)
    print(characters,"k")
    # Save the characters as individual image files (optional)
    for i, character in enumerate(characters):
        character_filename = f"character_{i}.png"
        cv2.imwrite(character_filename, character)

    print(time.time() - start)
    return characters

image = cv2.imread("/content/images.png")
out_image = east_detect(image)




[array([[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,   0, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,   0, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255,   0, 255, 255, 255,   0,
        255, 255, 255,   0, 255, 255, 255,   0,   0, 255, 255,   0, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [2

In [52]:
import numpy as np
from tensorflow.keras.models import load_model

# Load your pretrained model
model = load_model('/content/your_model (1).h5')

# binary_images is your array of binary images (MNIST-like)

print(out_image)
# Create an empty list to store predictions
predictions = []
desired_size = (28, 28)

# Resize the binary image to the desired size
resized_image = cv2.resize(binary_image, desired_size, interpolation=cv2.INTER_LINEAR)
# Loop through the binary images
for binary_image in out_image:
    # Ensure that the shape of the binary image matches the expected input shape of your model
    if binary_image.shape != (28, 28):
       resized_image = cv2.resize(binary_image, desired_size, interpolation=cv2.INTER_LINEAR)


    # Add a batch dimension (usually 1 image in the batch)
    input_image = np.expand_dims(resized_image, axis=0)

    # Make predictions using the model
    result = model.predict(input_image)

    predictions.append(result)



[array([[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,   0, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,   0, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255,   0, 255, 255, 255,   0,
        255, 255, 255,   0, 255, 255, 255,   0,   0, 255, 255,   0, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [2

In [53]:
import string

# Convert the output predictions to alphabets
def predictions_to_alphabets(predictions):
    alphabets = string.ascii_uppercase  # A-Z

    converted_predictions = []

    for prediction in predictions:
        max_prob_index = np.argmax(prediction)
        alphabet = alphabets[max_prob_index]
        converted_predictions.append(alphabet)

    return converted_predictions

In [54]:
predictions_to_alphabets(predictions)

['B', 'B']